In [453]:
#https://www.kaggle.com/datasets/ishikajohari/best-books-10k-multi-genre-data

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df = pd.read_csv("/Users/mattmacrides/Personal-Projects/Book Recommendation System/goodreads_data.csv")
#df = df[df['Book'].isin(["Apple Cider Vinegar Miracle Health System", "How to Kill a Rock Star"])]
#df = df.reset_index()
df.head(10)

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
5,5,The Little Prince,Antoine de Saint-Exupéry,A pilot stranded in the desert awakes one morn...,"['Classics', 'Fiction', 'Fantasy', 'Childrens'...",4.32,"1,924,063",https://www.goodreads.com/book/show/157993.The...
6,6,1984,George Orwell,The new novel by George Orwell is the major wo...,"['Classics', 'Fiction', 'Science Fiction', 'Dy...",4.19,"4,201,429",https://www.goodreads.com/book/show/61439040-1984
7,7,The Great Gatsby,F. Scott Fitzgerald,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"['Classics', 'Fiction', 'School', 'Historical ...",3.93,"4,839,642",https://www.goodreads.com/book/show/4671.The_G...
8,8,The Catcher in the Rye,J.D. Salinger,It's Christmas time and Holden Caulfield has j...,"['Classics', 'Fiction', 'Young Adult', 'Litera...",3.81,"3,315,881",https://www.goodreads.com/book/show/5107.The_C...
9,9,The Lord of the Rings,J.R.R. Tolkien,"One Ring to rule them all, One Ring to find th...","['Fantasy', 'Classics', 'Fiction', 'Adventure'...",4.52,"644,766",https://www.goodreads.com/book/show/33.The_Lor...


In [10]:
# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Parse the string representations into actual lists
gen = [ast.literal_eval(s) for s in df['Genres']]

# Fit and transform the 'genres' column
genre_matrix = mlb.fit_transform(gen)

# Create a DataFrame from the transformed data with custom column names
genre_df = pd.DataFrame(genre_matrix, columns=['is_' + genre for genre in mlb.classes_])

# Concatenate the new DataFrame with the original one
df = pd.concat([df, genre_df], axis=1)

# Replace missing (NaN) values in the 'Description' column with empty strings
df['Description'].fillna('', inplace=True)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features as needed

# Fit and transform the book descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Description'])

# Get the feature names (words) corresponding to the columns in the TF-IDF matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame from the TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Concatenate the new DataFrame with the original one
df = pd.concat([df, tfidf_df], axis=1)

# Replace missing (NaN) values in the 'Book' column with empty strings
df['Book'].fillna('', inplace=True)

# Initialize the TF-IDF vectorizer
book_tfidf_vectorizer = TfidfVectorizer()  # You can adjust the number of features as needed

# Fit and transform the book descriptions
book_tfidf_matrix = book_tfidf_vectorizer.fit_transform(df['Book'])

# Get the feature names (words) corresponding to the columns in the TF-IDF matrix
feature_names = book_tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame from the TF-IDF matrix
book_tfidf_df = pd.DataFrame(book_tfidf_matrix.toarray(), columns=feature_names)

# Concatenate the new DataFrame with the original one
df = pd.concat([df, book_tfidf_df], axis=1)

In [4]:
# Get the index of the two books: 9392 and 9620
df = df.iloc[[9392, 9620]]
df = df.reset_index()
df = df.iloc[:, 1:]
tfidf_df = tfidf_df.iloc[[9392, 9620]]
tfidf_df = tfidf_df.reset_index()
tfidf_df = tfidf_df.iloc[:, 1:]
book_tfidf_df = book_tfidf_df.iloc[[9392, 9620]]
book_tfidf_df = book_tfidf_df.reset_index()
book_tfidf_df = book_tfidf_df.iloc[:, 1:]
genre_df = genre_df.iloc[[9392, 9620]]
genre_df = genre_df.reset_index()
genre_df = genre_df.iloc[:, 1:]
df = df.reset_index()


In [5]:
genre_df

,is_12th Century,is_15th Century,is_16th Century,is_17th Century,is_18th Century,is_19th Century,is_20th Century,is_21st Century,is_Abuse,is_Academia,...,is_Writing,is_Young Adult,is_Young Adult Contemporary,is_Young Adult Fantasy,is_Young Adult Paranormal,is_Young Adult Romance,is_Young Readers,is_Zen,is_Zimbabwe,is_Zombies
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#ALL BOOKS
# Find columns with all values equal to 0.0
zero_columns = book_tfidf_df.columns[(book_tfidf_df == 0.0).all()]
# Remove the zero columns from the DataFrame
book_tfidf_df = book_tfidf_df.drop(columns=zero_columns)
book_tfidf_df

,apple,cider,health,how,kill,miracle,rock,star,system,to,vinegar
0,0.458377,0.414246,0.358226,0.000000,0.000000,0.364657,0.000000,0.000000,0.405465,0.000000,0.438849
1,0.000000,0.000000,0.000000,0.339685,0.521227,0.000000,0.521227,0.521227,0.000000,0.263783,0.000000


In [153]:
# Only 2 Books
# Find columns with all values equal to 0.0
#zero_columns = book_tfidf_df.columns[(book_tfidf_df == 0.0).all()]
# Remove the zero columns from the DataFrame
#book_tfidf_df = book_tfidf_df.drop(columns=zero_columns)
#book_tfidf_df

,apple,cider,health,how,kill,miracle,rock,star,system,to,vinegar
0,0.408248,0.408248,0.408248,0.000000,0.000000,0.408248,0.000000,0.000000,0.408248,0.000000,0.408248
1,0.000000,0.000000,0.000000,0.447214,0.447214,0.000000,0.447214,0.447214,0.000000,0.447214,0.000000


In [11]:
# Calculate the cosine similarity based on description data
description_sim_matrix = cosine_similarity(tfidf_df)
n_description_sim_matrix = 0.5 * (description_sim_matrix + 1)

# Calculate the cosine similarity based on genre data
genre_sim_matrix = cosine_similarity(genre_df)
n_genre_sim_matrix = 0.5 * (genre_sim_matrix + 1)

# Calculate the cosine similarity based on title data
book_sim_matrix = cosine_similarity(book_tfidf_df)
n_book_sim_matrix = book_sim_matrix#0.5 * (book_sim_matrix + 1)


def get_similar_books_with_similarity(book_title, num_recommendations, alpha, beta, gamma):
    book_index = df[df['Book'] == book_title].index[0]
    
    # Calculate similarity percentages for description
    desc_sim_scores = list(enumerate(n_description_sim_matrix[book_index]))
    desc_sim_scores = sorted(desc_sim_scores, key=lambda x: x[1], reverse=True)
    description_sim_scores = [i[1] for i in desc_sim_scores[1:num_recommendations + 1]]

    # Calculate similarity percentages for genre
    genre_sim_scores = list(enumerate(n_genre_sim_matrix[book_index]))
    genre_sim_scores = sorted(genre_sim_scores, key=lambda x: x[1], reverse=True)
    genre_sim_scores = [i[1] for i in genre_sim_scores[1:num_recommendations + 1]]

    # Calculate similarity percentages for book
    book_sim_scores = list(enumerate(n_book_sim_matrix[book_index]))
    book_sim_scores = sorted(book_sim_scores, key=lambda x: x[1], reverse=True)
    book_sim_scores = [i[1] for i in book_sim_scores[1:num_recommendations + 1]]

    # Combine the two similarity scores using alpha and beta weights
    similar_books_indices = [i[0] for i in desc_sim_scores[1:num_recommendations + 1]]  # Exclude the book itself
    similar_books = df['Book'].iloc[similar_books_indices]
    combined_similarity_scores = [
        (book, alpha * desc_sim + beta * genre_sim + gamma * book_sim, desc_sim, genre_sim, book_sim)
        for book, desc_sim, genre_sim, book_sim in zip(similar_books, description_sim_scores, genre_sim_scores, book_sim_scores)
    ]

    # Sort the combined similarity scores
    combined_similarity_scores = sorted(combined_similarity_scores, key=lambda x: x[1], reverse=True)

    # Create a DataFrame with the top N books and their similarity scores
    top_books_df = pd.DataFrame(combined_similarity_scores[:num_recommendations], columns=['Book', 'Overall', 'Description', 'Genre', 'Title'])
    
    return top_books_df


In [12]:
# Example usage:
book_title = "How to Kill a Rock Star"
similar_books_df = get_similar_books_with_similarity(book_title, num_recommendations=5, alpha=.4, beta=.4, gamma=.2)
similar_books_df.head()

,Book,Overall,Description,Genre,Title
0,Apple Cider Vinegar Miracle Health System,0.779248,0.814883,0.928571,0.409330
1,"Misty of Chincoteague (Misty, #1)",0.746026,0.737024,0.928571,0.398940
2,"Dune Messiah (Dune, #2)",0.740732,0.724597,0.928571,0.397323
3,Decline and Fall,0.727673,0.718851,0.928571,0.343517
4,Stillwell,0.726794,0.718743,0.928571,0.339341


I want to compare results if I only include the most popular columns

In [ ]:
#ALL: Apple Cider Vinegar Miracle Health System	    0.779248	0.814883	0.928571	0.409330
#JUST: Apple Cider Vinegar Miracle Health System	0.418786	0.546965	0.5	        0.0

Just mine these categories and find the similarity scores between them

In [7]:
df.columns

Index(['Unnamed: 0', 'Book', 'Author', 'Description', 'Genres', 'Avg_Rating',
       'Num_Ratings', 'URL', 'is_12th Century', 'is_15th Century',
       ...
       'ആട', 'ഉണ', 'ങള', 'ടന', 'മയ', 'യഴ', 'രങ', 'രണ', 'ഴയ', 'コンビニ人間'],
      dtype='object', length=81673)

ValueError: could not convert string to float: 'How to Kill a Rock Star'

In [9]:
newdf.head(10)

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,is_12th Century,is_15th Century,...,ആട,ഉണ,ങള,ടന,മയ,യഴ,രങ,രണ,ഴയ,コンビニ人間
9620,9620,How to Kill a Rock Star,Tiffanie DeBartolo,"Written in her wonderfully honest, edgy, passi...","['Romance', 'Contemporary', 'Music', 'Contempo...",4.06,"15,360",https://www.goodreads.com/book/show/113791.How...,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
